## Importing the Necessary Libraries

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

## Load the pretrained OD Model on COCO Dataset

In [2]:
model = tf.saved_model.load('./ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model')

# Load class names
class_names = {}
current_id = None
with open('./mscoco_complete_label_map.pbtxt', 'r') as f:
    for line in f:
        if "id:" in line:
            current_id = int(line.strip().split(' ')[-1])
        if "display_name:" in line:
            display_name = line.strip().split('"')[1]
            class_names[current_id] = display_name

### Function to Draw the box, using the Output Bounding Box Coords

In [3]:
def draw_boxes(frame, boxes, classes, scores, min_score_thresh=.5):
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    for i in range(boxes.shape[1]):
        if scores[0, i] > min_score_thresh:
            box = boxes[0, i] * np.array([frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]])
            class_id = classes[0, i]
            class_name = class_names.get(class_id, 'N/A')
            rect = Rectangle((box[1], box[0]), box[3] - box[1], box[2] - box[0], 
                             linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.text(box[1], box[0] - 10, class_name, color='r', fontsize=12, weight='bold')

    plt.axis('off')
    plt.show()

## **MAIN FUNCTION TO DETECT OBJECT**

In [4]:
def detect_objects(image_path):
    frame = cv2.imread(image_path)

    if frame is None:
        raise ValueError(f"Unable to read image at path: {image_path}")

    # Object detection
    input_tensor = tf.convert_to_tensor([frame], dtype=tf.uint8)
    detections = model(input_tensor)

    # Extracting detection results
    boxes = detections['detection_boxes'].numpy()
    classes = detections['detection_classes'].numpy().astype(np.int32)
    scores = detections['detection_scores'].numpy()

    # Draw boxes on the image
    draw_boxes(frame, boxes, classes, scores)

    # Create a list of detected objects
    detected_objects = []
    for i in range(boxes.shape[1]):
        if scores[0, i] > 0.5:  # threshold can be adjusted
            class_id = int(classes[0, i])
            class_name = class_names.get(class_id, 'N/A')
            detected_objects.append(class_name)

    return detected_objects

## Testing The Model

In [ ]:

image_path = './test/test1.jpg'
if not os.path.isfile(image_path):
    print("File does not exist")
else:
    print("File exists")
detected_objects = detect_objects(image_path)
print(f"Detected objects: {detected_objects}")